In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/RecFldGrn


# Read Raw Data

In [2]:
import os 
import pandas as pd

RecName = 'A1C'
folder = 'data/Simulations/'
raw_file_path =  os.path.join(folder, RecName) + '.csv'
print(raw_file_path)


data/Simulations/A1C.csv


In [3]:

# load all records
df = pd.read_csv(raw_file_path)
df

,PID,ECID,RID,V,DT
0,P0,EC13,5050,36.499557,2023-03-27
1,P0,EC14,4651,36.247254,2023-04-06
2,P0,EC18,5801,9.260195,2023-04-08
3,P0,EC87,3542,44.569271,2023-04-09
4,P0,EC201,5800,17.660021,2023-04-12
5,P1,EC33,5799,15.500044,2023-03-25
6,P1,EC0,4649,45.309869,2023-04-02
7,P1,EC26,6204,21.664842,2023-04-06
8,P1,EC2,3541,31.810007,2023-04-06
9,P1,EC28,3897,32.831454,2023-04-08


# Preprocess

## Add Missing Records

In [4]:
from recfldgrn.datapoint import load_df_data_from_folder

prefix_cols = ['PID', 'ECID']
focal_cols = ['RID']

Parent_RecName = 'EC'
rec_folder = 'data/ProcData/RecFolder/'
fullrec_folder =  os.path.join(rec_folder, Parent_RecName) 
df_parent = load_df_data_from_folder(fullrec_folder)
df_parent

,PID,ECID,DT_min,DT_max,BasicInfo
0,P0,EC18,2023-03-18,2023-04-16,D&X
1,P0,EC140,2023-03-19,2023-04-05,D&X
2,P0,EC23,2023-03-19,2023-04-16,C&W
3,P0,EC141,2023-03-20,2023-04-16,C&Z
4,P0,EC142,2023-03-20,2023-04-15,B&W
...,...,...,...,...,...
220,P6,EC15,2023-03-21,2023-04-09,B&X
221,P6,EC93,2023-03-23,2023-04-04,B&X
222,P6,EC195,2023-03-25,2023-03-25,C&Z
223,P6,EC173,2023-04-05,2023-04-13,C&W


In [5]:
import numpy as np

d = pd.merge(df_parent[prefix_cols], df, on = prefix_cols, how = 'left')
for col in focal_cols: 
    d.loc[d[col].isnull(), col] = np.ones(d[col].isnull().sum()).cumsum() + d[col].max()
    d[col] = d[col].astype(int)
    assert len(d[col]) == d[col].nunique()
    
df = d
df

,PID,ECID,RID,V,DT
0,P0,EC18,5801,9.260195,2023-04-08
1,P0,EC140,6205,NaN,NaN
2,P0,EC23,6206,NaN,NaN
3,P0,EC141,6207,NaN,NaN
4,P0,EC142,6208,NaN,NaN
...,...,...,...,...,...
220,P6,EC15,3546,16.714732,2023-04-09
221,P6,EC93,6401,NaN,NaN
222,P6,EC195,5056,46.861784,2023-03-25
223,P6,EC173,6402,NaN,NaN


## Update DT Format

In [6]:
df['DT'] = pd.to_datetime(df['DT'])

# Save To Disk by Bucket

In [7]:
from recfldgrn.datapoint import convert_PID_to_PIDgroup
from recfldgrn.datapoint import RANGE_SIZE, write_df_to_folders, load_df_data_from_folder

In [8]:
rec_folder = 'data/ProcData/RecFolder'
fullrec_folder = os.path.join(rec_folder, RecName)
print(df.shape, fullrec_folder)

if not os.path.exists(fullrec_folder): os.makedirs(fullrec_folder)
write_df_to_folders(RecName, fullrec_folder, df)


(225, 5) data/ProcData/RecFolder/A1C


In [9]:
from recfldgrn.datapoint import PatientDP
print(rec_folder)
PID = 'P4'
Pat = PatientDP(PID, rec_folder, RANGE_SIZE)
print(Pat)
dfx = Pat.get_df_rec(RecName)
dfx

data/ProcData/RecFolder


,PID,ECID,RID,V,DT
0,P4,EC125,6360,NaN,NaT
1,P4,EC24,6361,NaN,NaT
2,P4,EC36,5382,2.313767,2023-04-13
3,P4,EC80,6362,NaN,NaT
4,P4,EC163,6363,NaN,NaT
5,P4,EC106,6364,NaN,NaT
6,P4,EC85,6365,NaN,NaT
7,P4,EC88,6366,NaN,NaT
8,P4,EC179,6367,NaN,NaT
9,P4,EC48,6368,NaN,NaT


In [10]:
from recfldgrn.datapoint import load_df_data_from_folder
print(fullrec_folder)
df = load_df_data_from_folder(fullrec_folder)
df

data/ProcData/RecFolder/A1C


,PID,ECID,RID,V,DT
0,P0,EC18,5801,9.260195,2023-04-08
1,P0,EC140,6205,NaN,NaT
2,P0,EC23,6206,NaN,NaT
3,P0,EC141,6207,NaN,NaT
4,P0,EC142,6208,NaN,NaT
...,...,...,...,...,...
220,P6,EC15,3546,16.714732,2023-04-09
221,P6,EC93,6401,NaN,NaT
222,P6,EC195,5056,46.861784,2023-03-25
223,P6,EC173,6402,NaN,NaT


# Field & Grain

In [11]:
FieldGrainInfo = []

In [12]:
RecName = 'A1C'
prefix_ids = ['PID', 'ECID']
focal_ids = ['RID']
field_list = ['V', 'DT']

## Field `V`

In [13]:
# (a) field and (b) get field-focused df
field = 'V'
dfx = df[prefix_ids + focal_ids + [field]]
dfx

,PID,ECID,RID,V
0,P0,EC18,5801,9.260195
1,P0,EC140,6205,NaN
2,P0,EC23,6206,NaN
3,P0,EC141,6207,NaN
4,P0,EC142,6208,NaN
...,...,...,...,...
220,P6,EC15,3546,16.714732
221,P6,EC93,6401,NaN
222,P6,EC195,5056,46.861784
223,P6,EC173,6402,NaN


### Grain `'A1C@V-A1CNumeDftGrn'`

In [14]:
from recfldgrn.grains import Field2Grain_2_get_GrnStrList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnIdxList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnWgtList_Fn
from recfldgrn.utils import generate_grain_vocab_info

In [15]:
# (c) grain
grain = 'A1CNumeDftGrn'
field2grain = '-'.join([field, grain])
recfield2grain = RecName + '@' + field2grain
print(grain)
print(field2grain)
print(recfield2grain)

A1CNumeDftGrn
V-A1CNumeDftGrn
A1C@V-A1CNumeDftGrn


In [16]:
# grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
# s = dfx[field].apply(lambda x: grain_str_fn(x))
# s

In [17]:
# Vocab = generate_grain_vocab_info(s) # load from disk
# v2idx = Vocab['v2idx']
# v2idx

In [18]:
# grain_idx_fn = Field2Grain_2_get_GrnIdxList_Fn[recfield2grain]
# s = dfx[field].apply(lambda x: grain_idx_fn(x, v2idx))
# s

In [19]:
from recfldgrn.grains.misc.dt_grn import load_DTgrn_Dictionary
from transformers import AutoTokenizer

if 'Nume' in grain:
    # print('Current grain is a numeric grain')
    grain_wgt_fn = Field2Grain_2_get_GrnWgtList_Fn[recfield2grain]
    series_wgt = dfx[field].apply(grain_wgt_fn)
    Vocab = {'v2idx': len(series_wgt.iloc[0]) * [1]}
    v2idx = Vocab['v2idx']

elif 'DT' in grain:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = load_DTgrn_Dictionary()
    v2idx = Vocab['v2idx']
    
elif 'Tknz' in grain:
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    v2idx = tokenizer
    Vocab = {'v2idx': v2idx}

else:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = generate_grain_vocab_info(series_str)
    v2idx = Vocab['v2idx']

print(v2idx)
grain_idx_fn = Field2Grain_2_get_GrnIdxList_Fn[recfield2grain]
series_idx = dfx[field].apply(lambda x: grain_idx_fn(x, v2idx))
series_idx


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


0      [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
1      [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
2      [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
3      [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
4      [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
                             ...                        
220    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
221    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
222    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
223    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
224    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
Name: V, Length: 225, dtype: object

In [20]:
series_wgt

0      [0, 0, 0, 0, 0.7886271318696316, 1, 1.0, 1.0, ...
1      [1, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....
2      [1, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....
3      [1, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....
4      [1, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....
                             ...                        
220    [0, 0, 0, 0, 2.9184948708125233, 1, 1.0, 1.0, ...
221    [1, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....
222    [0, 0, 0, 0, 11.531938162947851, 1, 1.0, 1.0, ...
223    [1, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....
224    [0, 0, 0, 0, 3.0705198239387923, 1, 1.0, 1.0, ...
Name: V, Length: 225, dtype: object

In [21]:
d = {}
d['RecName'] = RecName

d['prefix_ids'] = prefix_ids
d['focal_ids'] = focal_ids 
d['field_list'] = field_list

d['field'] = field
d['grain'] = grain
d['field2grain'] = field2grain
d['recfield2grain'] = recfield2grain
d['Vocab'] = Vocab

FieldGrainInfo.append(d)

In [22]:
d

{'RecName': 'A1C',
 'prefix_ids': ['PID', 'ECID'],
 'focal_ids': ['RID'],
 'field_list': ['V', 'DT'],
 'field': 'V',
 'grain': 'A1CNumeDftGrn',
 'field2grain': 'V-A1CNumeDftGrn',
 'recfield2grain': 'A1C@V-A1CNumeDftGrn',
 'Vocab': {'v2idx': [1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1]}}

## Field `DT_min`

In [23]:
# (a) field and (b) get field-focused df
field = 'DT'
dfx = df[prefix_ids + focal_ids + [field]]
dfx

,PID,ECID,RID,DT
0,P0,EC18,5801,2023-04-08
1,P0,EC140,6205,NaT
2,P0,EC23,6206,NaT
3,P0,EC141,6207,NaT
4,P0,EC142,6208,NaT
...,...,...,...,...
220,P6,EC15,3546,2023-04-09
221,P6,EC93,6401,NaT
222,P6,EC195,5056,2023-03-25
223,P6,EC173,6402,NaT


### AgeNumeGrn

In [24]:
# (c) grain
grain = 'DTDftGrn'
field2grain = '-'.join([field, grain])
recfield2grain = RecName + '@' + field2grain
print(grain)
print(field2grain)
print(recfield2grain)

DTDftGrn
DT-DTDftGrn
A1C@DT-DTDftGrn


In [25]:
from recfldgrn.grains.misc.dt_grn import load_DTgrn_Dictionary
from transformers import AutoTokenizer

if 'Nume' in grain:
    # print('Current grain is a numeric grain')
    grain_wgt_fn = Field2Grain_2_get_GrnWgtList_Fn[recfield2grain]
    series_wgt = dfx[field].apply(grain_wgt_fn)
    Vocab = {'v2idx': len(series_wgt.iloc[0]) * [1]}
    v2idx = Vocab['v2idx']

elif 'DT' in grain:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = load_DTgrn_Dictionary()
    v2idx = Vocab['v2idx']
    
elif 'Tknz' in grain:
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    v2idx = tokenizer
    Vocab = {'v2idx': v2idx}

else:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = generate_grain_vocab_info(series_str)
    v2idx = Vocab['v2idx']

print(v2idx)
grain_idx_fn = Field2Grain_2_get_GrnIdxList_Fn[recfield2grain]
series_idx = dfx[field].apply(lambda x: grain_idx_fn(x, v2idx))
series_idx


{'_padding': 0, '_missing': 1, 'Yx': 2, 'Y2010': 3, 'Y2011': 4, 'Y2012': 5, 'Y2013': 6, 'Y2014': 7, 'Y2015': 8, 'Y2016': 9, 'Y2017': 10, 'Y2018': 11, 'Y2019': 12, 'Y2020': 13, 'Y2021': 14, 'Y2022': 15, 'Y2023': 16, 'Y2024': 17, 'Y2025': 18, 'Y2026': 19, 'Y2027': 20, 'Y2028': 21, 'Y2029': 22, 'Mx': 23, 'M1': 24, 'M2': 25, 'M3': 26, 'M4': 27, 'M5': 28, 'M6': 29, 'M7': 30, 'M8': 31, 'M9': 32, 'M10': 33, 'M11': 34, 'M12': 35, 'Dx': 36, 'D1': 37, 'D2': 38, 'D3': 39, 'D4': 40, 'D5': 41, 'D6': 42, 'D7': 43, 'D8': 44, 'D9': 45, 'D10': 46, 'D11': 47, 'D12': 48, 'D13': 49, 'D14': 50, 'D15': 51, 'D16': 52, 'D17': 53, 'D18': 54, 'D19': 55, 'D20': 56, 'D21': 57, 'D22': 58, 'D23': 59, 'D24': 60, 'D25': 61, 'D26': 62, 'D27': 63, 'D28': 64, 'D29': 65, 'D30': 66, 'D31': 67, 'Sx': 68, 'S1': 69, 'S2': 70, 'S3': 71, 'S4': 72, 'Wx': 73, 'W1': 74, 'W2': 75, 'W3': 76, 'W4': 77, 'W5': 78, 'W6': 79, 'W7': 80, 'WYx': 81, 'WY1': 82, 'WY2': 83, 'WY3': 84, 'WY4': 85, 'WY5': 86, 'WY6': 87, 'WY7': 88, 'WY8': 89, 'WY

0      [16, 27, 44, 70, 79, 96, 138]
1       [2, 23, 36, 68, 73, 81, 137]
2       [2, 23, 36, 68, 73, 81, 137]
3       [2, 23, 36, 68, 73, 81, 137]
4       [2, 23, 36, 68, 73, 81, 137]
                   ...              
220    [16, 27, 45, 70, 80, 96, 138]
221     [2, 23, 36, 68, 73, 81, 137]
222    [16, 26, 61, 69, 79, 94, 138]
223     [2, 23, 36, 68, 73, 81, 137]
224    [16, 27, 42, 70, 77, 96, 138]
Name: DT, Length: 225, dtype: object

In [26]:
d = {}
d['RecName'] = RecName

d['prefix_ids'] = prefix_ids
d['focal_ids'] = focal_ids 
d['field_list'] = field_list


d['field'] = field
d['grain'] = grain
d['field2grain'] = field2grain
d['recfield2grain'] = recfield2grain
d['Vocab'] = Vocab

FieldGrainInfo.append(d)

# Save Field Grain Info



In [27]:
df_FieldGrainInfo = pd.DataFrame(FieldGrainInfo)
df_FieldGrainInfo

,RecName,prefix_ids,focal_ids,field_list,field,grain,field2grain,recfield2grain,Vocab
0,A1C,"[PID, ECID]",[RID],"[V, DT]",V,A1CNumeDftGrn,V-A1CNumeDftGrn,A1C@V-A1CNumeDftGrn,"{'v2idx': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
1,A1C,"[PID, ECID]",[RID],"[V, DT]",DT,DTDftGrn,DT-DTDftGrn,A1C@DT-DTDftGrn,"{'idx2v': {0: '_padding', 1: '_missing', 2: 'Y..."


In [28]:
fldgrn_folder = 'data/ProcData/FldGrnInfo'

if not os.path.exists(fldgrn_folder): os.makedirs(fldgrn_folder)

fullfldgrn_file = os.path.join(fldgrn_folder, RecName + '.p')
df_FieldGrainInfo.to_pickle(fullfldgrn_file)